<a href="https://colab.research.google.com/github/ryuuTAylor/caltech101-clip-classification/blob/main/caltech101_clip_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/ryuuTAylor/caltech101-clip-classification.git
%cd caltech101-clip-classification

Cloning into 'caltech101-clip-classification'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), done.
/content/caltech101-clip-classification


In [4]:
!pip install transformers torch torchvision lightning wandb
import torch
import torchvision
import transformers
import lightning
import wandb
wandb.login(key="a646c68496bc8916e4338a0447e80256838a8134")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 25.9 MB/s eta 0:00:00


**Part I : Collect the required DataSet**

In [5]:
from torchvision.datasets import Caltech101
from torchvision.transforms import ToTensor, Compose, Resize, Normalize, Lambda
from torch.utils.data import DataLoader

In [6]:
# Download and Load the Dataset
transform = Compose([
    Lambda(lambda img: img.convert("RGB")),  # Ensure all images are RGB
    Resize((224, 224)),  # Resize images to CLIP's expected input size
    ToTensor(),  # Convert images to tensors
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

dataset = Caltech101(root="./data", download=True, transform=transform)

Downloading...
From (original): https://drive.google.com/uc?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp
From (redirected): https://drive.usercontent.google.com/download?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp&confirm=t&uuid=8e5d6669-7c0d-437e-9a8c-98614d072f27
To: /content/caltech101-clip-classification/data/caltech101/101_ObjectCategories.tar.gz
100%|██████████| 132M/132M [00:02<00:00, 44.4MB/s]


Extracting ./data/caltech101/101_ObjectCategories.tar.gz to ./data/caltech101


Downloading...
From: https://drive.google.com/uc?id=175kQy3UsZ0wUEHZjqkUDdNVssr7bgh_m
To: /content/caltech101-clip-classification/data/caltech101/Annotations.tar
100%|██████████| 14.0M/14.0M [00:00<00:00, 62.3MB/s]


Extracting ./data/caltech101/Annotations.tar to ./data/caltech101


In [7]:
# Split Dataset into Training and Testing
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [8]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

**Part II : Use CLIP to Extract Embeddings**

In [20]:
from transformers import CLIPProcessor, CLIPModel

# Load pre-trained CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

clip_model.to("cuda")

# Keep CLIP model frozen
clip_model.eval()

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [23]:
from PIL import Image

def extract_embeddings(image_batch):
    with torch.no_grad():
        # Undo normalization: convert from [-1, 1] to [0, 255]
        image_batch = [(img * 0.5 + 0.5).mul(255).byte() for img in image_batch]

        # Convert tensor images to PIL format for CLIP processor
        image_batch = [Image.fromarray(img.permute(1, 2, 0).cpu().numpy()) for img in image_batch]

        # Prepare images for CLIP
        inputs = clip_processor(images=image_batch, return_tensors="pt", padding=True)

        # Ensure inputs are on the GPU
        inputs = {key: value.to(clip_model.device) for key, value in inputs.items()}

        # Extract image features
        embeddings = clip_model.get_image_features(**inputs)
        return embeddings

In [16]:
# Test the Embedding Extraction
for images, labels in train_loader:
    embeddings = extract_embeddings(images)
    print("Embedding shape:", embeddings.shape)
    break

Embedding shape: torch.Size([32, 512])


**Part III : Build the Lightweight Classifier**

In [12]:
# Classify using Multi-Layer Perceptron (MLP)
import torch.nn as F

class LightweightClassifier(F.Module):
    def __init__(self, input_dim=512, num_classes=101):
        super().__init__()
        self.fc = F.Sequential(
            F.Linear(input_dim, 256),  # Hidden layer
            F.ReLU(),                 # Activation function
            F.Dropout(0.3),           # Dropout for regularization
            F.Linear(256, num_classes)  # Output layer
        )

    def forward(self, x):
        return self.fc(x)

**Part IV : Integrate with PyTorch Lightning**

In [21]:
import pytorch_lightning as pl

class CLIPClassifier(pl.LightningModule):
    def __init__(self, classifier):
        super().__init__()
        self.classifier = classifier
        self.loss_fn = F.CrossEntropyLoss()

    def forward(self, x):
        return self.classifier(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        # Move images and labels to the GPU
        images, labels = images.to(self.device), labels.to(self.device)

        # Extract embeddings on the GPU
        embeddings = extract_embeddings(images)
        logits = self(embeddings)
        loss = self.loss_fn(logits, labels)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        images, labels = images.to(self.device), labels.to(self.device)
        embeddings = extract_embeddings(images)
        logits = self(embeddings)
        loss = self.loss_fn(logits, labels)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.classifier.parameters(), lr=1e-3)

**Part V : Initialize and Train the Model**

In [24]:
# Initialize the lightweight classifier
classifier = LightweightClassifier()

# Wrap it in a LightningModule
model = CLIPClassifier(classifier)

# # Define the trainer
# trainer = pl.Trainer(
#     max_epochs=10,  # Set the number of epochs
#     accelerator="gpu",  # Use GPU
#     devices=1,  # Number of GPUs to use
#     log_every_n_steps=10
# )

# # Train the model
# trainer.fit(model, train_loader, test_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                  | Params | Mode 
-------------------------------------------------------------
0 | classifier | LightweightClassifier | 157 K  | train
1 | loss_fn    | CrossEntropyLoss      | 0      | train
-------------------------------------------------------------
157 K     Trainable params
0         Non-trainable params
157 K     Total params
0.629     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


**Part VI : Log Metrics with wandb**

In [25]:
from pytorch_lightning.loggers import WandbLogger

# Initialize wandb logger
wandb_logger = WandbLogger(project="clip-caltech101")

# Update the trainer to use wandb
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu",
    devices=1,
    logger=wandb_logger,
    log_every_n_steps=10
)

# Train the model with wandb logging
trainer.fit(model, train_loader, test_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
